# Importing Libraries

In [7]:
from pathlib import Path
import pandas as pd
import numpy as np
import nltk
import re
#importing stopwords is optional, in this case it decreased accuracy
#from nltk.corpus import stopwords
import itertools
import time
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Timo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
start_time = time.time()

# Load DataSet

In [9]:
data = pd.read_csv('C:/Users/Timo/Source/Repos/NLP/NLP_ChatBot/datasets/sentiment_analysis/dataset.csv')

#data

# Lemmatization Of DataSet

In [10]:
#stopset = set(stopwords.words('english'))

from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

# Pre-Processing of Data

In [11]:

def cleaning(text):
    txt = str(text)
    txt = re.sub(r"http\S+", "", txt)
    if len(txt) == 0:
        return 'no text'
    else:
        txt = txt.split()
        index = 0
        for j in range(len(txt)):
            if txt[j][0] == '@':
                index = j
        txt = np.delete(txt, index)
        if len(txt) == 0:
            return 'no text'
        else:
            words = txt[0]
            for k in range(len(txt)-1):
                words+= " " + txt[k+1]
            txt = words
            txt = re.sub(r'[^\w]', ' ', txt)
            if len(txt) == 0:
                return 'no text'
            else:
                txt = ''.join(''.join(s)[:2] for _, s in itertools.groupby(txt))
                txt = txt.replace("'", "")
                txt = nltk.tokenize.word_tokenize(txt)
                #data.content[i] = [w for w in data.content[i] if not w in stopset]
                for j in range(len(txt)):
                    txt[j] = lem.lemmatize(txt[j], "v")
                if len(txt) == 0:
                    return 'no text'
                else:
                    return txt

# Construction Of Words Dictionary

In [ ]:
data['text'] = data['text'].map(lambda x: cleaning(x))
#data
data = data.reset_index(drop=True)
for i in range(len(data)):
    words = data.text[i][0]
    for j in range(len(data.text[i])-1):
        words+= ' ' + data.text[i][j+1]
    data.text[i] = words

C:\Users\Timo\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Training of Data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.model_selection import train_test_split
exit(0)
x_train, x_test, y_train, y_test = train_test_split(data.text, data.label, test_size=0.25, random_state=0)

x_train = x_train.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)

y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

vectorizer = TfidfVectorizer(min_df=3, max_df=0.9)

train_vectors = vectorizer.fit_transform(x_train)
test_vectors = vectorizer.transform(x_test)

model = svm.SVC(kernel='linear') 
model.fit(train_vectors, y_train) 
predicted_sentiment = model.predict(test_vectors)

print(classification_report(y_test, predicted_sentiment))
predicted_sentiments = []
for s in range(len(predicted_sentiment)):
    predicted_sentiments.append(predicted_sentiment[s])

path = Path('C:/Users/Timo/Source/Repos/NLP/NLP_ChatBot/datasets/emotion_recognizer/dataset.csv').resolve()
prediction_df = pd.DataFrame({'Content':x_test, 'Emotion_predicted':predicted_sentiment, 'Emotion_actual': y_test})
prediction_df.to_csv(path, index = False)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

# Accuracy 

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

cm = confusion_matrix(predicted_sentiment, y_test)
print(cm)

print(accuracy_score(y_test, predicted_sentiment))

# Test File

In [ ]:
prediction_df